In [29]:
from pyiem.plot import figure
from pyiem.util import get_dbconn

ASOS = get_dbconn("asos")
cursor = ASOS.cursor()

cursor.execute("""
 select distinct date(valid) from alldata where station = 'OMA' 
 and array_to_string(wxcodes, '') LIKE '%%FZRA%%' and
 extract(hour from ((valid + '10 minutes'::interval) at time zone 'UTC')) in (23, 0, 1, 2)
 and extract(month from valid) in (11, 12,1,2, 3, 4) ORDER by date ASC
""")
wdates = []
for row in cursor:
    wdates.append(row[0])

ASOS.close()
print(wdates)

[datetime.date(1949, 2, 20), datetime.date(1951, 2, 15), datetime.date(1956, 11, 22), datetime.date(1959, 12, 27), datetime.date(1960, 2, 4), datetime.date(1961, 2, 17), datetime.date(1961, 3, 5), datetime.date(1962, 2, 8), datetime.date(1968, 12, 18), datetime.date(1973, 1, 21), datetime.date(1975, 11, 29), datetime.date(1979, 1, 18), datetime.date(1980, 12, 6), datetime.date(1982, 1, 24), datetime.date(1982, 11, 27), datetime.date(1982, 12, 27), datetime.date(1983, 11, 26), datetime.date(1989, 1, 25), datetime.date(1989, 3, 14), datetime.date(1989, 12, 26), datetime.date(1991, 12, 19), datetime.date(1992, 12, 28), datetime.date(1993, 2, 11), datetime.date(1996, 1, 17), datetime.date(1996, 3, 24), datetime.date(1997, 12, 21), datetime.date(1999, 1, 22), datetime.date(2001, 2, 23), datetime.date(2001, 3, 11), datetime.date(2002, 3, 8), datetime.date(2005, 3, 18), datetime.date(2008, 11, 10), datetime.date(2008, 12, 18), datetime.date(2009, 1, 9), datetime.date(2010, 12, 15), datetime.d

In [ ]:
POSTGIS = get_dbconn("postgis")
cursor = POSTGIS.cursor()

cursor.execute("""
 select f.valid, p.smps, p.height from raob_profile p JOIN raob_flights f on (p.fid = f.fid) 
 WHERE f.station in ('KOAX','KOMA','KOVN') and p.smps * 2.23 > 30 and p.levelcode = 9 
 and extract(hour from f.valid at time zone 'UTC') = 0 ORDER by valid DESC
""")
dates = []
winds = []
for row in cursor:
    print(row)
    winds.append(row[1] * 2.23)
    dates.append(row[0])

POSTGIS.close()

In [20]:
import datetime

# day - 1 :)
wdates.append(datetime.date(2023, 1, 2))

In [25]:
import numpy as np

dbconn = get_dbconn("raob")
cursor = dbconn.cursor()

ar_whght = []
ar_tmpc = []
ar_dwpc = []
ar_dates = []
for date in wdates:
    if date.year < 1947:
        continue
    cursor.execute(
        """
    select height, tmpc, dwpc, smps * 2.23 from raob_profile_%s p JOIN raob_flights f on 
     (p.fid = f.fid) where f.station in ('KOAX','KOMA','KOVN')  and 
     p.height < 7000 and tmpc >= -100 and dwpc >= -100 and 
     valid = '%s 00:00+00'::timestamptz  ORDER by height ASC
    """
        % (date.year, (date + datetime.timedelta(days=1)).strftime("%Y-%m-%d"))
    )
    h = []
    t = []
    d = []
    for row in cursor:
        h.append(row[0])
        t.append(row[1])
        d.append(row[2])
    if len(h) == 0:
        print(f"No data for {date}")
        continue
    print(date, cursor.rowcount, t[0], h[0])
    ar_whght.append(np.array(h))
    ar_tmpc.append(np.array(t))
    ar_dwpc.append(np.array(d))
    ar_dates.append(date)

No data for 1949-02-20
No data for 1951-02-15
No data for 1956-11-22
No data for 1959-12-27
1960-02-04 15 -0.6 403.0
1961-02-17 18 -6.0 403.0
1961-03-05 16 -1.0 403.0
1962-02-08 15 -2.0 403.0
1968-12-18 16 0.0 403.0
1973-01-21 18 -1.7 403.0
1975-11-29 25 -2.8 400.0
1979-01-18 25 -2.6 400.0
1980-12-06 22 -1.1 400.0
1982-01-24 21 -12.0 400.0
1982-11-27 22 0.4 400.0
1982-12-27 23 -0.4 400.0
1983-11-26 19 -2.6 400.0
1989-01-25 23 -1.1 400.0
1989-03-14 23 -1.8 400.0
1989-12-26 21 -1.1 400.0
1991-12-19 33 -1.8 400.0
1992-12-28 33 -3.9 400.0
1993-02-11 27 -2.7 432.0
No data for 1996-01-17
1996-03-24 39 -1.3 350.0
1997-12-21 24 -0.8 350.0
1999-01-22 22 -0.5 350.0
2001-02-23 30 -4.0 350.0
2001-03-11 19 -1.7 350.0
2002-03-08 15 -1.1 350.0
2005-03-18 12 0.0 350.0
2008-11-10 16 0.4 350.0
2008-12-18 41 -5.7 350.0
2009-01-09 18 -4.1 350.0
2010-12-15 25 -3.3 350.0
2015-11-26 31 -2.7 350.0
2015-11-28 29 -1.3 350.0
2017-01-15 21 -0.9 350.0
2019-02-22 37 0.2 350.0
2020-04-02 24 -1.5 350.0
2022-12-19 20 

In [46]:
def compute_mean(h, s):
    """Compute a mean profile"""
    x = []
    y = []
    for hght in range(300, 4301, 200):
        vals = []
        for myh, mys in zip(h, s):
            for myhh, myss in zip(myh, mys):
                if myhh >= hght and myhh < (hght + 200):
                    vals.append(myss)
        x.append(sum(vals) / float(len(vals)))
        y.append(hght + 100)
    return x, y


import matplotlib.pyplot as plt

title = "00 UTC Omaha Sounding on %s Dates of 22-02 UTC KOMA FZRA Reports" % (
    len(ar_dwpc) - 1,
)
fig = figure(
    title=title,
    subtitle="Confined to November thru April, 3 Jan 2023 was producing rain",
    figsize=(8, 6),
)
ax = [
    fig.add_axes([0.1, 0.1, 0.35, 0.75]),
    fig.add_axes([0.6, 0.1, 0.35, 0.75]),
]
for i, (h, s) in enumerate(zip(ar_whght, ar_tmpc)):
    if max(s) > 100:
        continue
    # n = np.array(s) / s[0]
    ax[0].plot(s, h, color="tan")

ax[0].plot(
    ar_tmpc[-1], ar_whght[-1], color="r", lw=2, zorder=2, label="3 Jan 2023"
)
x, y = compute_mean(ar_whght, ar_tmpc)
ax[0].plot(x, y, color="k", lw=2, zorder=2, label="Composite")
ax[0].legend(ncol=2, loc=4)

for i, (h, s, d) in enumerate(zip(ar_whght, ar_tmpc, ar_dates)):
    maxt = np.max(s)
    hght = h[np.argmax(s)]
    print(d)
    label = None
    if i == 0:
        label = "All Cases"
    if d.year == 2023:
        label = "3 Jan 2023"
    ax[1].scatter(
        maxt, hght, color=("r" if d.year == 2023 else "tan"), label=label
    )

ax[1].legend(ncol=2, loc=4)

# for i, (h,s) in enumerate(zip(ar_whght,ar_dwpc)):
#    if max(s) > 100:
#        continue
# n = np.array(s) / s[0]
#    ax[1].plot(s,h, color='tan')
# ax[1].plot(ar_dwpc[-1], ar_whght[-1], color='r', lw=2, zorder=2, label='9 Feb 2021')

# x, y = compute_mean(ar_whght, ar_dwpc)
# ax[1].plot(x,y, color='k', lw=2, zorder=2, label='Composite')
# ax[1].legend(ncol=2, loc=4)

ax[0].text(
    0.01,
    1.02,
    "Profile",
    transform=ax[0].transAxes,
    bbox=dict(edgecolor="tan", facecolor="#EEEEEE"),
)
ax[1].text(
    0.02,
    1.02,
    "Height of Max Air Temp",
    transform=ax[1].transAxes,
    bbox=dict(edgecolor="tan", facecolor="#EEEEEE"),
)
ax[1].grid(True)
ax[0].grid(True)
ax[0].set_ylim(0, 4000)
ax[1].set_ylim(0, 4000)
ax[0].set_xlim(-20, 14)
ax[1].set_xlim(-20, 14)
ax[0].set_xlabel(r"Air Temperature $^\circ$C")
ax[1].set_xlabel(r"Air Temperature $^\circ$C")
ax[0].set_ylabel("Height Above Sea Level [m]")
ax[1].set_ylabel("Height Above Sea Level [m]")
fig.savefig("230103.png")
plt.close()

1960-02-04
1961-02-17
1961-03-05
1962-02-08
1968-12-18
1973-01-21
1975-11-29
1979-01-18
1980-12-06
1982-01-24
1982-11-27
1982-12-27
1983-11-26
1989-01-25
1989-03-14
1989-12-26
1991-12-19
1992-12-28
1993-02-11
1996-03-24
1997-12-21
1999-01-22
2001-02-23
2001-03-11
2002-03-08
2005-03-18
2008-11-10
2008-12-18
2009-01-09
2010-12-15
2015-11-26
2015-11-28
2017-01-15
2019-02-22
2020-04-02
2022-12-19
2023-01-02
